In [21]:
import numpy as np
import matlab.engine
from datetime import date
import os
import scipy.io as sio
import pickle as pkl
import matplotlib.pyplot as plt
import os
from Sanity import check_dataset
from tqdm import tqdm
from data_utils import save_datafiles, plot_histogram

Parameters

In [22]:
sanity_check = True
OTA_data = True
Frame_802_11 = True
N_captures = 40
save_data = False

Create Folder

In [23]:
if save_data:
    id = "QPSK_BL_104_OTA"

    if OTA_data == True and Frame_802_11 == False:
        raise Exception("OTA Data must be Framed")

    current_date = date.today()
    file_name = "Datasets/Dataset_" + str(id) + "_" + str(current_date)

    # make a folder for today's data
    try:
        try:
            os.mkdir("Datasets")
        except:
            pass
        os.mkdir(file_name)
        os.mkdir(file_name + "/Figures")
    except:
        pass

Generate TX

In [24]:
eng = matlab.engine.start_matlab()

In [25]:
if Frame_802_11:
    eng.TX_data_generate("802_11 Framed", nargout=0)
else:
    eng.TX_data_generate("Raw Data", nargout=0)

In [ ]:
if save_data:
    Data_Input, Encoder_Output, Receiver_Output, Data_Output, Frame_Error = (
        [],
        [],
        [],
        [],
        [],
    )

for i in tqdm(range(N_captures)):

    if OTA_data:
        os.system("python3 flow_graph_RX.py")
    else:
        eng.awgn_channel("complex_noise", nargout=0)

    # Decode RX
    if Frame_802_11:
        eng.RX_data_extract("802_11 Framed", nargout=0)
    else:
        eng.RX_data_extract("Raw Data", nargout=0)

    if save_data:
        # Prepare Data
        # Input Data
        Data = sio.loadmat("Data_Files/Data_Input.mat")
        Data_Input.append(Data["Data_Input"])

        # Encoder Output
        Data = sio.loadmat("Data_Files/Encoded_Output.mat")
        Encoder_Output.append(Data["Encoder_Output"])

        # Receiver Output
        Data = sio.loadmat("Data_Files/Receiver_Output.mat")
        Receiver_Output.append(Data["Receiver_Output"])

        # Output LLR
        Data = sio.loadmat("Data_Files/Data_Output.mat")
        Data_Output.append(Data["Data_Output"])

        # Frame Error
        Data = sio.loadmat("Data_Files/Frame_Error.mat")
        Frame_Error.append(Data["Frame_Error"])